This is a pure Tensorflow implementation of AlexNet, This still doesn't work as it should and gets very low (~ 0.1 Top 1 accuracy). It will be reviewed later.

In [13]:
import _pickle as cPickle
import os
import sys
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import time
from keras.utils import to_categorical,plot_model
%matplotlib inline

Initializes needed variables and imports data set.

In [14]:
batch_size=128
image_width=32
image_height=32
image_depth=3
learning_rate=0.01
def load_cifar10(num_training=49000, num_validation=1000, num_test=10000):
    """
    Fetch the CIFAR-10 dataset from the web and perform preprocessing to prepare
    it for the two-layer neural net classifier. These are the same steps as
    we used for the SVM, but condensed to a single function.
    """
    # Load the raw CIFAR-10 dataset and use appropriate data types and shapes
    cifar10 = tf.keras.datasets.cifar10.load_data()
    (X_train, y_train), (X_test, y_test) = cifar10
    X_train = np.asarray(X_train, dtype=np.float32)
    y_train = np.asarray(y_train, dtype=np.int32).flatten()
    X_test = np.asarray(X_test, dtype=np.float32)
    y_test = np.asarray(y_test, dtype=np.int32).flatten()

    # Subsample the data
    mask = range(num_training, num_training + num_validation)
    X_val = X_train[mask]
    y_val = y_train[mask]
    mask = range(num_training)
    X_train = X_train[mask]
    y_train = y_train[mask]
    mask = range(num_test)
    X_test = X_test[mask]
    y_test = y_test[mask]

    # Normalize the data: subtract the mean pixel and divide by std
    mean_pixel = X_train.mean(axis=(0, 1, 2), keepdims=True)
    std_pixel = X_train.std(axis=(0, 1, 2), keepdims=True)
    X_train = (X_train - mean_pixel) / std_pixel
    X_val = (X_val - mean_pixel) / std_pixel
    X_test = (X_test - mean_pixel) / std_pixel

    return X_train, y_train, X_val, y_val, X_test, y_test


# Invoke the above function to get our data.
X_train, y_train, X_val, y_val, X_test, y_test = load_cifar10()       
print('Train data shape: ', X_train.shape)
print('Train labels shape: ', y_train.shape, y_train.dtype)
print('Validation data shape: ', X_val.shape)
print('Validation labels shape: ', y_val.shape)
print('Test data shape: ', X_test.shape)
print('Test labels shape: ', y_test.shape)
y_train = to_categorical(y_train)
y_test = to_categorical(y_test)
y_val = to_categorical(y_val)

Train data shape:  (49000, 32, 32, 3)
Train labels shape:  (49000,) int32
Validation data shape:  (1000, 32, 32, 3)
Validation labels shape:  (1000,)
Test data shape:  (10000, 32, 32, 3)
Test labels shape:  (10000,)


In [15]:
device = '/device:CPU:0'
print_every = 100
print('Using device: ', device)

Using device:  /device:CPU:0


All layers use as default a gaussian weight initialization with std 0.01 and a mean of 0.0. The biases are of default 0.0 but for some layers they are 1.0 as specified in AlexNet

In [16]:
def convRelu(x,kernelSize,outputNum, strideSize = 1,bias = 0.0,std = 0.01,activation = tf.nn.relu,padding='SAME'):
    inputSize = x.get_shape().as_list()[-1]
    initializer = tf.contrib.layers.xavier_initializer()
    weights = tf.Variable(initializer([kernelSize,kernelSize,inputSize,outputNum]))
    bias = tf.Variable(tf.constant(bias, shape=[outputNum]), dtype=tf.float32)
    Initial = tf.nn.conv2d(x,weights,strides = [1, strideSize, strideSize, 1], padding = padding)
    Initial = tf.reshape(tf.nn.bias_add(Initial,bias),tf.shape(Initial))
    if activation:
        Initial = activation(Initial)
    return Initial

def maxPool(X,filterHeight,filterWidth,strideX,strideY,padding='SAME'):
    return tf.nn.max_pool(X,[1,filterHeight,filterWidth,1],[1,strideY,strideX,1],padding)
def lrn(X,radius=2,bias= 1.0 ,alpha = 0.0001,beta = 0.75):
    return tf.nn.local_response_normalization(X,radius,bias,alpha,beta)
def dropout(x,prob):
    return tf.nn.dropout(x,prob)
def FullyConnected(x,outputSize, bias = 0.0, std = 0.01,relu = True):
    inputSize = x.get_shape().as_list()
    initializer = tf.contrib.layers.xavier_initializer()
    if len(inputSize) == 4:
        weights = tf.Variable(initializer([inputSize[1] * inputSize[2] * inputSize[3], outputSize]))
        x = tf.reshape(x, [-1, weights.get_shape().as_list()[0]])
    else:
        weights = tf.Variable(initializer([inputSize[-1], outputSize]))
    bias =  tf.Variable(tf.constant(bias, shape=[outputSize], dtype=tf.float32))
    Initial =  tf.matmul(x,weights)
    Initial = tf.nn.bias_add(Initial,bias)
    if relu:
        Initial = tf.nn.relu(Initial)
    return Initial

In [17]:
def buildCNN(Input):
        con1 = convRelu(Input,3,96,2,padding='SAME')
        pool1 = maxPool(con1,2,2,2,2,padding='SAME') 
        lrn1 = lrn(pool1)
    
        con2 = convRelu(lrn1, 5,256,1)
        pool2 = maxPool(con2,3,3,2,2,padding='SAME') 
        lrn2 = lrn(pool2)
        
        con3 = convRelu(lrn2,3,384,1)
        con4 = convRelu(con3, 3,384,1)
        con5 = convRelu(con4, 3,256,1)
        pool5 = maxPool(con5, 3, 3, 2, 2, padding='SAME')
        FC1 = FullyConnected(pool5,4096,bias = 1.0,relu = True)
        FC1 = dropout(FC1,0.5)
        FC2 = FullyConnected(FC1,4096,bias = 1.0, relu=True)
        FC2 = dropout(FC2,0.5)
        FC3 = FullyConnected(FC2,10,bias = 1.0,relu = False)
        return FC3
def ModelLoss(logits,labels):
     return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits = logits, labels = labels))
def ModelAccuracy(logits,labels):
    return tf.reduce_mean(tf.cast(tf.equal(tf.argmax(logits, 1), tf.argmax(labels, 1)), tf.float32))

There is probably something wrong with this function, still have to figure out what i did wrong.

In [20]:
def train():
    with tf.device(device):
        x = tf.placeholder(tf.float32, [None, 32, 32, 3])
        label = tf.placeholder(tf.int32, [None,10])
        learningRate = tf.placeholder("float", None, name='learning_rate')
    logits = buildCNN(x)
    accuracy = ModelAccuracy(logits,label)
    loss = ModelLoss(logits,label)
    Optimizer = tf.train.GradientDescentOptimizer(learningRate).minimize(loss)
    init = tf.global_variables_initializer()
    sess = tf.Session()
    sess.run(init)
    
    
    i = 0 
    trainLen = len(X_train)
    train_range = list(zip(range(0, len(X_train), batch_size), range(batch_size, len(X_train), batch_size)))
    if len(X_train) % batch_size > 0:
         train_range.append((train_range[-1][1], len(X_train)))
    cur_learning_rate = 0.01
    for epoch in range(30):
        j = 0
        if epoch % 10 == 0 and epoch > 0:
            cur_learning_rate /= 10
        for start, end in train_range:
            train_x = X_train[start:end]
            train_y = y_train[start:end] 
            _, loss_result = sess.run([Optimizer, loss], feed_dict={x: train_x, label: train_y,learningRate: cur_learning_rate})
            accuracy_result, loss_result = sess.run([accuracy, loss],feed_dict={x: X_val, label: y_val})
            print((j*batch_size),"/",trainLen, "Epoch:",epoch," loss:",loss_result," accuracy:",accuracy_result,end="\r")
            j += 1
        i += 1

In [21]:
train()

KeyboardInterrupt: 